# Does the Tourism Preferences Changes after COVID19?

## CASA0013: Foundations of Spatial Data Science

### Student Ids: ucftXXXX, ucftYYYY, etc.

## Green Space Data

In [1]:
# Import visualisation modules
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt 

#Import modules
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import os

import warnings 
warnings.simplefilter(action='ignore')
ox.__version__

'1.2.2'

In [2]:
# Set up query
q1 = {
    "tourism":"camp_site",
    'leisure':'common',
    'leisure':'dog_park',
    'natural':'scrub',
    'natural':'fell',
    'landuse':'forest',
    'leisure':'garden',
    'landuse':'greenfield',
    'leisure':'golf_course',
    'landuse':'grass',
    'natural':'grassland',
    'natural':'heath',
    'landuse':'meadow',
    'natural':'moor',
    'leisure':'nature_reserve',
    'landuse':'orchard',
    'leisure':'park',
    'leisure':'pitch',
    'landuse':'recreation_ground',
    'landuse':'village_green',
    'landuse':'vineyard',
    'natural':'wood'
    }

greenspace = ox.geometries.geometries_from_place(
            'Greater London, UK',
            tags = q1,
            which_result=1)

greenspace.head(5)

geometry created_by barrier bicycle  \
element_type osmid                                                              
node         20851184    POINT (-0.33622 51.40443)        NaN     NaN     NaN   
             92273182    POINT (-0.40698 51.48916)       JOSM     NaN     NaN   
             895874399   POINT (-0.23028 51.55593)        NaN     NaN     NaN   
             920063079   POINT (-0.06894 51.56576)        NaN     NaN     NaN   
             1296074660  POINT (-0.17313 51.41807)        NaN     NaN     NaN   

                        foot source leisure                    name  \
element_type osmid                                                    
node         20851184    NaN    NaN   pitch  The Royal Tennis Court   
             92273182    NaN    NaN     NaN                     NaN   
             895874399   NaN    NaN   pitch      Kilburn Cosmos RFC   
             920063079   NaN    NaN   pitch                     NaN   
             1296074660  NaN    NaN   pitch                     NaN   

                             name:ru         sport  ... oneway construction  \
element_type osmid                                  ...                       
node         20851184    Реал-теннис   real_tennis  ...    NaN          NaN   
             92273182            NaN           NaN  ...    NaN          NaN   
             895874399           NaN         rugby  ...    NaN          NaN   
             920063079           NaN           NaN  ...    NaN          NaN   
             1296074660          NaN  table_tennis  ...    NaN          NaN   

                        proposed religion danger genus:en informal ways type  \
element_type osmid                                                             
node         20851184        NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             92273182        NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             895874399       NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             920063079       NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             1296074660      NaN      NaN    NaN      NaN      NaN  NaN  NaN   

                        network  
element_type osmid               
node         20851184       NaN  
             92273182       NaN  
             895874399      NaN  
             920063079      NaN  
             1296074660     NaN  

[5 rows x 278 columns]

In [3]:
greenspace.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 15452 entries, ('node', 20851184) to ('relation', 15268904)
Columns: 278 entries, geometry to network
dtypes: geometry(1), object(277)
memory usage: 33.4+ MB


In [4]:
path = os.path.join('data')

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
greenspace.to_csv(os.path.join(path,'greenspace.csv'), index=False)

## Tourism Attraction Data

In [5]:
# Set up query
q2 = {'tourism':'attraction'}

# Run query
# Jin: I change ox.pois.pois_from_place to ox.geometries.geometries_from_place, for the adjustment of osmnx version change.
# details: https://stackoverflow.com/questions/71559143/what-happened-to-the-pois-module-in-osmnx-and-what-to-use-now
tourism_attraction = ox.geometries.geometries_from_place(
            'Greater London, UK',
            tags = q2,
            which_result=1)

tourism_attraction.head(5)

historic                      name     tourism  \
element_type osmid                                                       
node         25524252   building           Blewcoat School  attraction   
             26559743        NaN                       NaN  attraction   
             252602371       NaN  London Bridge Experience  attraction   
             269236138       NaN      Little Holland House  attraction   
             293221901       NaN    Hall Place and Gardens  attraction   

                        wikidata                        wikipedia  \
element_type osmid                                                  
node         25524252   Q4926413               en:Blewcoat School   
             26559743        NaN                              NaN   
             252602371  Q7748032  en:The London Bridge Experience   
             269236138       NaN                              NaN   
             293221901  Q5642615                              NaN   

                                         geometry access barrier bicycle  \
element_type osmid                                                         
node         25524252   POINT (-0.13606 51.49830)    NaN     NaN     NaN   
             26559743   POINT (-0.14525 51.39520)    NaN     NaN     NaN   
             252602371  POINT (-0.08826 51.50639)    NaN     NaN     NaN   
             269236138  POINT (-0.17065 51.35530)    NaN     NaN     NaN   
             293221901   POINT (0.16023 51.44819)    NaN     NaN     NaN   

                       place  ... int_name source:description name:ban  \
element_type osmid            ...                                        
node         25524252    NaN  ...      NaN                NaN      NaN   
             26559743    NaN  ...      NaN                NaN      NaN   
             252602371   NaN  ...      NaN                NaN      NaN   
             269236138   NaN  ...      NaN                NaN      NaN   
             293221901   NaN  ...      NaN                NaN      NaN   

                       name:eo name:hak name:mai name:pms name:tl name:sw  \
element_type osmid                                                          
node         25524252      NaN      NaN      NaN      NaN     NaN     NaN   
             26559743      NaN      NaN      NaN      NaN     NaN     NaN   
             252602371     NaN      NaN      NaN      NaN     NaN     NaN   
             269236138     NaN      NaN      NaN      NaN     NaN     NaN   
             293221901     NaN      NaN      NaN      NaN     NaN     NaN   

                       name:xmf  
element_type osmid               
node         25524252       NaN  
             26559743       NaN  
             252602371      NaN  
             269236138      NaN  
             293221901      NaN  

[5 rows x 359 columns]

In [6]:
tourism_attraction.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 344 entries, ('node', 25524252) to ('relation', 12942436)
Columns: 359 entries, historic to name:xmf
dtypes: geometry(1), object(358)
memory usage: 976.9+ KB


In [7]:
path = os.path.join('data')

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
tourism_attraction.to_csv(os.path.join(path,'tourism_attraction.csv'), index=False)

## Airbnb data (Pre-precessing)

My workflow planning of this:

Aim: calculate the availablity for each Airbnb listing in 2019 and 2022.

    Count f or t in field "available" by rows and also group by "listing_id".
    
    Get a field we need finally, which represent the number of days booked/occupied in the period of one year.
    
    Join this field to listing.csv by listing_id.

### read in data

In [8]:
IA_cal_2022 = pd.read_csv('./data/calendar.csv.gz',compression='gzip', low_memory=False)
IA_cal_2022

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,13913,2022-09-11,f,$50.00,$50.00,1.0,29.0
1,106332,2022-09-11,f,$55.00,$55.00,3.0,365.0
2,106332,2022-09-12,f,$55.00,$55.00,3.0,365.0
3,106332,2022-09-13,f,$55.00,$55.00,3.0,365.0
4,106332,2022-09-14,f,$55.00,$55.00,3.0,365.0
...,...,...,...,...,...,...,...
25310907,554172169432589107,2023-09-07,f,$134.00,$134.00,3.0,5.0
25310908,554172169432589107,2023-09-08,f,$134.00,$134.00,3.0,4.0
25310909,554172169432589107,2023-09-09,f,$134.00,$134.00,3.0,3.0
25310910,554172169432589107,2023-09-10,f,$134.00,$134.00,2.0,1125.0


### fixing data type

In [9]:
#check data type, and find the column "available" is object type. so change it to boolean?
IA_cal_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25310912 entries, 0 to 25310911
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.3+ GB


In [10]:
bools = ['available']
IA_cal_2022.sample(5, random_state=43)[bools]

,available
9656045,t
9976390,f
4219951,t
9336946,f
22511373,f


In [11]:
# to map 't' and 'f' to True and False
for b in bools:
    print(f"Converting {b}")
    IA_cal_2022[b] = IA_cal_2022[b].replace({'f':False, 't':True}).astype('bool')

Converting available


In [12]:
IA_cal_2022.sample(5, random_state=43)[bools]

,available
9656045,True
9976390,False
4219951,True
9336946,False
22511373,False


In [13]:
# check that it has been transformed to boolean value
IA_cal_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25310912 entries, 0 to 25310911
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       bool   
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: bool(1), float64(2), int64(1), object(3)
memory usage: 1.2+ GB


### count T/F by rows

In [14]:
# split a test dataset first, because the original whole table is toooooo large.
# p.s. What is a feather file? Can I transfer csv.gz to feather to make the memory usage smaller?
df_test = IA_cal_2022.iloc[:500,:]  
df_test

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,13913,2022-09-11,False,$50.00,$50.00,1.0,29.0
1,106332,2022-09-11,False,$55.00,$55.00,3.0,365.0
2,106332,2022-09-12,False,$55.00,$55.00,3.0,365.0
3,106332,2022-09-13,False,$55.00,$55.00,3.0,365.0
4,106332,2022-09-14,False,$55.00,$55.00,3.0,365.0
...,...,...,...,...,...,...,...
495,13913,2023-06-07,True,$50.00,$50.00,1.0,29.0
496,13913,2023-06-08,True,$50.00,$50.00,1.0,29.0
497,13913,2023-06-09,True,$50.00,$50.00,1.0,29.0
498,13913,2023-06-10,True,$51.00,$51.00,1.0,29.0


In [15]:
df_test.groupby('listing_id')['available'].sum()
# yes, it works.

listing_id
13913     252
106332      2
Name: available, dtype: int64

In [17]:
%%time
# count the number of True values for each listing ID.
# I get about ... 
avlblt_2022 = IA_cal_2022.groupby('listing_id')['available'].sum()  # avlblt: availability

CPU times: user 422 ms, sys: 120 ms, total: 542 ms
Wall time: 831 ms


In [24]:
avlblt_2022

listing_id
13913                 343
15400                  70
17402                 249
24328                 116
25123                   0
                     ... 
712328309550998941    349
712328479212931623    339
712365421277069988    365
712383057902799115    361
712395062732173290    353
Name: available, Length: 69348, dtype: int64